In [1]:
# Update sklearn to prevent version mismatches
#!pip install sklearn --upgrade

In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
#!pip install joblib

In [3]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

koi_disposition  koi_fpflag_nt  koi_fpflag_ss  koi_fpflag_co  koi_fpflag_ec  \
0       CONFIRMED              0              0              0              0   
1  FALSE POSITIVE              0              1              0              0   
2  FALSE POSITIVE              0              1              0              0   
3       CONFIRMED              0              0              0              0   
4       CONFIRMED              0              0              0              0   

   koi_period  koi_period_err1  koi_period_err2  koi_time0bk  \
0   54.418383     2.479000e-04    -2.479000e-04   162.513840   
1   19.899140     1.490000e-05    -1.490000e-05   175.850252   
2    1.736952     2.630000e-07    -2.630000e-07   170.307565   
3    2.525592     3.760000e-06    -3.760000e-06   171.595550   
4    4.134435     1.050000e-05    -1.050000e-05   172.979370   

   koi_time0bk_err1  ...  koi_steff_err2  koi_slogg  koi_slogg_err1  \
0          0.003520  ...             -81      4.467           0.064   
1          0.000581  ...            -176      4.544           0.044   
2          0.000115  ...            -174      4.564           0.053   
3          0.001130  ...            -211      4.438           0.070   
4          0.001900  ...            -232      4.486           0.054   

   koi_slogg_err2  koi_srad  koi_srad_err1  koi_srad_err2         ra  \
0          -0.096     0.927          0.105         -0.061  291.93423   
1          -0.176     0.868          0.233         -0.078  297.00482   
2          -0.168     0.791          0.201         -0.067  285.53461   
3          -0.210     1.046          0.334         -0.133  288.75488   
4          -0.229     0.972          0.315         -0.105  296.28613   

         dec  koi_kepmag  
0  48.141651      15.347  
1  48.134129      15.436  
2  48.285210      15.597  
3  48.226200      15.509  
4  48.224670      15.714  

[5 rows x 41 columns]

# Select your features (columns)

In [5]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_fpflag_nt','koi_fpflag_ss','koi_fpflag_co','koi_fpflag_ec','koi_period','koi_time0bk',
                        'koi_impact','koi_duration','koi_depth','koi_prad','koi_teq','koi_insol','koi_model_snr',
                        'koi_tce_plnt_num','koi_steff','koi_slogg','koi_srad','ra','dec','koi_kepmag']]
feature_names = selected_features.columns
feature_names

Index(['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec',
       'koi_period', 'koi_time0bk', 'koi_impact', 'koi_duration', 'koi_depth',
       'koi_prad', 'koi_teq', 'koi_insol', 'koi_model_snr', 'koi_tce_plnt_num',
       'koi_steff', 'koi_slogg', 'koi_srad', 'ra', 'dec', 'koi_kepmag'],
      dtype='object')

# Create a Train Test Split

Use `koi_disposition` for the y values

In [6]:
# Set disposition for y
target = df["koi_disposition"]
target_names = ["CANDIDATE", "CONFIRMED", "FALSE POSITIVE"]

data = target.copy()
data_binary_encoded = pd.get_dummies(data)
data_binary_encoded.head()

CANDIDATE  CONFIRMED  FALSE POSITIVE
0          0          1               0
1          0          0               1
2          0          0               1
3          0          1               0
4          0          1               0

In [7]:
# Test split data, test data = 20%
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(selected_features, data_binary_encoded, test_size=0.40, random_state=42, stratify=data_binary_encoded)
#X_train, X_test, y_train, y_test = train_test_split(selected_features, target, test_size=0.40, random_state=42)

In [8]:
X_train.head()

koi_fpflag_nt  koi_fpflag_ss  koi_fpflag_co  koi_fpflag_ec  koi_period  \
1768              0              0              0              0    2.215713   
6077              0              0              1              1    3.556435   
3855              0              0              0              0  240.799047   
1664              0              0              0              0    8.630501   
1072              0              1              0              0   42.943863   

      koi_time0bk  koi_impact  koi_duration  koi_depth  koi_prad  koi_teq  \
1768   131.821070       0.796        1.8211      285.1      1.70     1283   
6077   133.569600       0.146        8.5290       45.8      0.75     1297   
3855   334.140370       0.647       16.6660     5022.9     10.31      342   
1664   139.574380       0.595        3.1440      325.8      1.43      739   
1072   155.206515       1.285        3.7591     9207.9     41.62      520   

      koi_insol  koi_model_snr  koi_tce_plnt_num  koi_steff  koi_slogg  \
1768     638.56           22.1                 1       5417      4.447   
6077     669.38           13.4                 1       5831      4.289   
3855       3.22          170.1                 1       5758      4.168   
1664      70.63           14.9                 1       5352      4.599   
1072      17.26          273.0                 1       5750      4.400   

      koi_srad         ra        dec  koi_kepmag  
1768     0.901  297.02008  43.432549      15.392  
6077     1.109  297.16602  46.199188      13.874  
3855     1.385  296.66791  49.461861      14.822  
1664     0.764  289.37704  40.001160      15.868  
1072     0.940  290.16513  51.467258      14.708

In [9]:
y_train.head()

CANDIDATE  CONFIRMED  FALSE POSITIVE
1768          0          1               0
6077          0          0               1
3855          0          1               0
1664          0          1               0
1072          0          0               1

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [10]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

C:\Users\j6921\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [11]:
X_train_scaled

array([[0.        , 0.        , 0.        , ..., 0.78501035, 0.43822704,
        0.74798047],
       [0.        , 0.        , 1.        , ..., 0.79168504, 0.61508864,
        0.61322681],
       [0.        , 0.        , 0.        , ..., 0.76890355, 0.82365997,
        0.69738127],
       ...,
       [1.        , 0.        , 0.        , ..., 0.83898827, 0.67030402,
        0.4750111 ],
       [0.        , 1.        , 0.        , ..., 0.30642662, 0.66355466,
        0.37976032],
       [0.        , 0.        , 0.        , ..., 0.70883178, 0.76315601,
        0.78331114]])

In [12]:
print(f"{X_train_scaled.max()} {X_train_scaled.min()}")
print(f"{y_train.max()} {y_train.min()}") 

1.0 0.0
CANDIDATE         1
CONFIRMED         1
FALSE POSITIVE    1
dtype: uint8 CANDIDATE         0
CONFIRMED         0
FALSE POSITIVE    0
dtype: uint8


In [13]:
X_test_scaled

array([[1.        , 0.        , 0.        , ..., 0.60407561, 0.54061565,
        0.60053262],
       [1.        , 0.        , 0.        , ..., 0.6623646 , 0.87957516,
        0.73200178],
       [0.        , 1.        , 0.        , ..., 0.57143164, 0.37179971,
        0.41695517],
       ...,
       [0.        , 1.        , 0.        , ..., 0.84733186, 0.51385345,
        0.6689747 ],
       [1.        , 0.        , 1.        , ..., 0.80852864, 0.26056501,
        0.79795828],
       [0.        , 1.        , 0.        , ..., 0.77780146, 0.63722652,
        0.65051043]])

# Train the Model



In [14]:
#from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train_scaled, y_train)
#print(f"Training Data Score: {model2.score(X_train_scaled, y_train)}")
#print(f"Testing Data Score: {model2.score(X_test_scaled, y_test)}")
a = rf.score(X_train_scaled, y_train)
b = rf.score(X_test_scaled, y_test)
print(f"{a}, {b}")

1.0, 0.8798712906685735


In [15]:
rf

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [16]:
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.13155018729974843, 'koi_fpflag_co'),
 (0.13043648281934306, 'koi_model_snr'),
 (0.11996761261689308, 'koi_fpflag_nt'),
 (0.09733167025471166, 'koi_fpflag_ss'),
 (0.07924871370820591, 'koi_prad'),
 (0.05277540710138989, 'koi_depth'),
 (0.04721511273998067, 'koi_period'),
 (0.043055620230661715, 'koi_fpflag_ec'),
 (0.039172117571447826, 'koi_impact'),
 (0.03336649065669032, 'koi_duration'),
 (0.03269662395229906, 'koi_teq'),
 (0.029724498076703267, 'koi_insol'),
 (0.029695936309658045, 'koi_time0bk'),
 (0.022648345212781834, 'koi_steff'),
 (0.021861905573197223, 'koi_slogg'),
 (0.02152141120106521, 'ra'),
 (0.020389925202979987, 'koi_kepmag'),
 (0.019937419114708244, 'koi_srad'),
 (0.018831185583567462, 'dec'),
 (0.008573334773967291, 'koi_tce_plnt_num')]

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [17]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
param_grid = { 
    'n_estimators': [50, 100, 200, 400, 600],
    'max_features': ['auto', 'sqrt', 'log2']
}
grid = GridSearchCV(rf, param_grid, verbose=3)

In [18]:
# Train the model with GridSearch
grid.fit(X_train, y_train)

Fitting 3 folds for each of 15 candidates, totalling 45 fits
[CV] max_features=auto, n_estimators=50 ..............................


C:\Users\j6921\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  max_features=auto, n_estimators=50, score=0.8726752503576538, total=   0.2s
[CV] max_features=auto, n_estimators=50 ..............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV]  max_features=auto, n_estimators=50, score=0.8791130185979972, total=   0.2s
[CV] max_features=auto, n_estimators=50 ..............................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.5s remaining:    0.0s


[CV]  max_features=auto, n_estimators=50, score=0.8984263233190272, total=   0.2s
[CV] max_features=auto, n_estimators=100 .............................
[CV]  max_features=auto, n_estimators=100, score=0.871244635193133, total=   0.4s
[CV] max_features=auto, n_estimators=100 .............................
[CV]  max_features=auto, n_estimators=100, score=0.8876967095851216, total=   0.4s
[CV] max_features=auto, n_estimators=100 .............................
[CV]  max_features=auto, n_estimators=100, score=0.8991416309012875, total=   0.4s
[CV] max_features=auto, n_estimators=200 .............................
[CV]  max_features=auto, n_estimators=200, score=0.8755364806866953, total=   0.9s
[CV] max_features=auto, n_estimators=200 .............................
[CV]  max_features=auto, n_estimators=200, score=0.8876967095851216, total=   0.9s
[CV] max_features=auto, n_estimators=200 .............................
[CV]  max_features=auto, n_estimators=200, score=0.9027181688125894, total=   

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:  1.2min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [50, 100, 200, 400, 600], 'max_features': ['auto', 'sqrt', 'log2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [19]:
print(grid.best_params_)
print(grid.best_score_)

{'max_features': 'sqrt', 'n_estimators': 400}
0.8900810681926562


In [20]:
rf_tuned = RandomForestClassifier(n_estimators=100, max_features='log2')
rf_tuned = rf_tuned.fit(X_train_scaled, y_train)
#print(f"Training Data Score: {model2.score(X_train_scaled, y_train)}")
#print(f"Testing Data Score: {model2.score(X_test_scaled, y_test)}")
a_tuned = rf_tuned.score(X_train_scaled, y_train)
b_tuned = rf_tuned.score(X_test_scaled, y_test)
print(f"{a_tuned}, {b_tuned}")

1.0, 0.8809438684304612


# Save the Model

In [21]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'your_name.sav'
joblib.dump(your_model, filename)

NameError: name 'your_model' is not defined